In [5]:
# For the data loader
import os
#import cv2
from torch.utils.data import Dataset, DataLoader

# For data augmentation
from torchvision import transforms

# For creating the model
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

# For training the model
import numpy as np

# For tracking the model
import wandb

# Step 1: Load the data (dataloader)

In [7]:
# The SeaIceData class contains functions for loading the data and creating a custom dataset
class SeaIceDataset(Dataset):
    def __init__(self, sar_path: str, chart_path: str, transform=None, augmentation=None):
        self.sar_path = sar_path
        self.sar_files = os.listdir(self.sar_path)
        self.chart_path = chart_path
        self.chart_files = os.listdir(self.chart_path)
        self.transform = transform
        ### apply augmentation at the data loader stage or later stage (for training data only?) ###
        # self.augmentation = augmentation
        # self.train_dataloader =  train_dataloader

# Fetch the data
    def __getitem__(self, index):
        sar_img = os.path.join(self.sar_path, self.sar_files[index])
        chart_img = os.path.join(self.chart_path, self.chart_files[index]) 

        ### Could/should we use cv2 or io for reading the data?
        
        sar = io.imread(sar_img).copy()  # take all bands for shape of 256 x 256 x 3
        chart = io.imread(chart_img).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
        chart[chart < 80] = 0  # binarise to water
        chart[chart >= 80] = 255  # binarise to ice
        sample = {"sar": sar, "chart": chart}

        if self.transform:
            sample = {"sar": self.transform(sar), "chart": self.transform(chart).squeeze(0).long()}
        return sample

        ### TBC if needed at this stage ###
        #if self.augmentation:
            #sample = {"sar": self.augmentation(sar), "chart": self.augmentation(chart)}

# Keep going through all data    
    def __len__(self):
        return len(self.sar_files)

### TBC if needed at this stage - Transformations/augmentation for training data only ###
#train_transform = transforms.Compose([
#    transforms.RandomHorizontalFlip(),
#    transforms.RandomRotation(30)
#])


# Step 2: Data Augmentation

See: https://github.com/qubvel/segmentation_models.pytorch/blob/master/examples/cars%20segmentation%20(camvid).ipynb

# Step 3: Split dataset into training/testing/validation

In [ ]:
# split the data into training and validation sets
train_data = SARData(sar_dir, chart_dir, train_transform, train=True)
val_data = SARData(sar_dir, chart_dir, val_transform, train=False)

# create dataloaders for the training and validation sets
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=False)

### Example from Andrew
train_dataset = SeaIceDataset(sar_path="./sar", chart_path="./chart", transform=transforms.ToTensor())
train_dataloader = DataLoader(train_dataset, shuffle=True)
val_dataset = SeaIceDataset(sar_path="./sar", chart_path="./chart", transform=transforms.ToTensor())
val_dataloader = DataLoader(val_dataset, shuffle=False)

### From https://github.com/qubvel/segmentation_models.pytorch/blob/master/examples/cars%20segmentation%20(camvid).ipynb

train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)


### EXAMPLE FROM https://github.com/MohammadBakir/Pytorch-Flower-Image-Classification/blob/master/Image%20Classifier%20Densenet201.py
#Defining data directories, modify accordingly
data_dir = './flower_data/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

#Define transforms for the training and validation sets
#Using pretrained Pytorch model trained on image sizes 224. Modify ResizedCrop and CenterCrop according to needs. 
data_transforms_train = transforms.Compose([transforms.RandomResizedCrop(256),
                                            transforms.RandomRotation(30),
                                            transforms.ColorJitter(),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.CenterCrop(224), 
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_transforms_validation = transforms.Compose([transforms.Resize(256),
                                                 transforms.CenterCrop(224),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_transforms_test = transforms.Compose([transforms.Resize(256),
                                                 transforms.CenterCrop(224),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#Load the datasets with ImageFolder
image_dataset_train = datasets.ImageFolder(train_dir, transform = data_transforms_train)

image_dataset_validation = datasets.ImageFolder(valid_dir, transform = data_transforms_validation)

image_dataset_test = datasets.ImageFolder(test_dir, transform = data_transforms_validation)


# Using the image datasets and the trainforms, define the dataloaders
#batch size and num workers can be modified accordingly. 
batch_size =256
num_workers=4
 

dataloader_train = torch.utils.data.DataLoader(image_dataset_train, batch_size=batch_size,
                                               num_workers=num_workers, shuffle=True)

dataloader_valid = torch.utils.data.DataLoader(image_dataset_validation, batch_size=batch_size,
                                               num_workers=num_workers, shuffle=True)

dataloader_test = torch.utils.data.DataLoader(image_dataset_test, batch_size=batch_size,
                                               num_workers=num_workers, shuffle=True)

# Step 4: Create the model

In [9]:
### See https://pytorch.org/hub/pytorch_vision_densenet/ for info about Densenet

### See https://paperswithcode.com/lib/torchvision/densenet for documentation on how to set this up!!!!

#### MOST HELPFUL SO FAR ####
## 1. Work through this: https://github.com/shounak8/AIML_Tutotials/blob/master/Deep_Learning/PyTorch/santa_or_not/santa_pytorch_pretrained_model.ipynb 
## 2. Then this: https://www.kaggle.com/code/balraj98/unet-with-pretrained-resnet50-encoder-pytorch
## 3. Ot this: https://github.com/MohammadBakir/Pytorch-Flower-Image-Classification/blob/master/Image%20Classifier%20Densenet201.py 

# Using Densetnet201 from https://segmentation-modelspytorch.readthedocs.io/en/latest/ 
densenet = smp.Unet('densenet201', encoder_weights='imagenet', pretrained=True)

### The model has been pre-trained on XXX so we need to transform our data to match

### Densenet expects 256 size?



ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['car']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)



# create a custom classifier using the densenet201 encoder
class Densenet(nn.Module):
    def __init__(self):
        super(Densenet, self).__init__()
        self.encoder = smp.models.densenet201(pretrained=True)
        num_ftrs = self.encoder.classifier.in_features
        self.encoder.classifier = nn.Linear(num_ftrs, 1)
    
    def forward(self, x):
        x = self.encoder(x)
        return x
    
# initialize the classifier model
model = Densenet()

# move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


TypeError: __init__() got an unexpected keyword argument 'pretrained'

Train the model

In [ ]:
import numpy as np

# set the number of epochs for training
num_epochs = 50

# keep track of training loss for each epoch
train_loss_history = []

# start training
for epoch in range(num_epochs):
    # set the model to training mode
    model.train()
    
    # keep track of training loss for this epoch
    epoch_loss = 0
    
    # loop through the training data
    for (chart_img, sar_img), target in train_loader:
        # move data to GPU if available
        chart_img = chart_img.to(device)
        sar_img = sar_img.to(device)
        target = target.to(device)
        
        # clear gradients
        optimizer.zero_grad()
        
        # forward pass
        output = model(sar_img)
        
        # calculate loss
        loss = criterion(output, target.float().unsqueeze(1))
        
        # backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # accumulate loss for this epoch
        epoch_loss += loss.item()
        
    # average loss for this epoch
    epoch_loss = epoch_loss / len(train_loader)
    train_loss_history.append(epoch_loss)
    
    # print loss for this epoch
    print("Epoch {}/{} - Train Loss: {:.6f}".format(epoch+1, num_epochs, epoch_loss))
    
# save the trained model
torch.save(model.state_dict(), "trained_model.pth")


Run & evaluate model performance

In [ ]:
## From Andrew

trainer.fit(segmenter, train_dataloader, val_dataloader)

Tune hyperparameters

In [ ]:
import torch
import numpy as np
import segmentation_models_pytorch as smp
from PIL import Image

# Load the SAR image and the labelled ice chart
sar_image = np.array(Image.open("sar_image.tif"))
ice_chart = np.array(Image.open("ice_chart.tif"))

# Resize and normalize the images
resize = (256, 256)
sar_image = np.array(Image.fromarray(sar_image).resize(resize))
sar_image = sar_image.astype(np.float32) / 65535.0 # TIFF images are typically 16-bit, so normalize to [0, 1]
ice_chart = np.array(Image.fromarray(ice_chart).resize(resize))
ice_chart = ice_chart.astype(np.float32) / 255.0

# Load the UNet model with a ResNet34 encoder and pre-trained ImageNet weights
model = smp.Unet('resnet34', encoder_weights='imagenet')

# Prepare the SAR image for inference
sar_image_tensor = torch.from_numpy(sar_image).to(torch.float32).unsqueeze(0).unsqueeze(0) # add batch and channel dimensions
preds = model.predict(sar_image_tensor)

# Threshold the predicted segmentation mask
threshold = 0.5
binary_preds = (preds > threshold).astype(np.uint8)

# Evaluate the binary mask
accuracy = np.mean(binary_preds == ice_chart)
print("Accuracy:", accuracy)


An example from https://github.com/shounak8/AIML_Tutotials/blob/master/Deep_Learning/PyTorch/santa_or_not/santa_pytorch_pretrained_model.ipynb 

In [11]:
import torch
import numpy as np
import segmentation_models_pytorch as smp
from PIL import Image
import tifffile as tiff

# Load the SAR image and the labelled ice chart
sar_image = np.array(tiff.imread("/Users/meghanplumridge/Desktop/S1B_EW_GRDM_1SDH_20181210T085100_20181210T085208_013977_019EF9_260F.tif"))
ice_chart = np.array(tiff.imread("20181209.tiff"))

# Resize and normalize the images
resize = (256, 256)
sar_image = np.array(Image.fromarray(sar_image).resize(resize))
sar_image = sar_image.astype(np.float32) / 65535.0 # TIFF images are typically 16-bit, so normalize to [0, 1]
ice_chart = np.array(Image.fromarray(ice_chart).resize(resize))
ice_chart = ice_chart.astype(np.float32) / 255.0

# Load the UNet model with a ResNet34 encoder and pre-trained ImageNet weights
model = smp.Unet('densenet201', encoder_weights='imagenet')

# Replace the final layer with a binary classification layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Prepare the data for training
sar_image_tensor = torch.from_numpy(sar_image).to(torch.float32).permute(2, 0, 1).unsqueeze(0) # add batch and channel dimensions
ice_chart_tensor = torch.from_numpy(ice_chart).to(torch.float32).unsqueeze(0).unsqueeze(0) # add batch and channel dimensions

# Train the model
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(sar_image_tensor)
    loss = criterion(outputs, ice_chart_tensor)
    loss.backward()
    optimizer.step()

# Evaluate the model on the test data
model.eval()
with torch.no_grad():
    preds = model(sar_image_tensor)
    binary_preds = (preds > 0.5).squeeze().cpu().numpy().astype(np.uint8)

# Evaluate the binary mask
accuracy = np.mean(binary_preds == ice_chart)
print("Accuracy:", accuracy)


TypeError: Cannot handle this data type: (1, 1, 3), <f4

In [ ]:
## IMPORT THE MODULES ##
import os
import torch
import numpy as np
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from skimage import io
from torchvision import transforms
from torch.nn import Linear, CrossEntropyLoss
from pytorch_lightning import Trainer



### LOAD THE DATA : ATTEMPT 1 ##

## Load the SAR image as training AND testing data for this example
#sar_image_train = np.array(Image.open("sar_image.tif"))
#sar_image_test = np.array(Image.open("sar_image.tif"))
## Load the ice chart for evaluation of model performance
#ice_chart = np.array(Image.open("ice_chart.tif"))

#### LOAD THE DATA : ATTEMPT 2 ###
## In the directories are files called sar_image.tif and the corresponding ice_chart.tif
#sar_path = "/Users/meghanplumridge/Desktop/CNN_sar_data/"
#sar_files = os.listdir(sar_path)
#chart_path = "/Users/meghanplumridge/Desktop/CNN_chart_data/"
#chart_files = os.listdir(chart_path)
#sar_name = os.path.join(sar_path, sar_files[i])
#chart_name = os.path.join(chart_path, chart_files[i])

#### LOAD THE DATA : ATTEMPT 3 ###
sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


sar = io.imread(sar_name).copy()  # take all bands for shape of 256 x 256 x 3
chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
chart[chart < 80] = 0  # binarise to water
chart[chart >= 80] = 255  # binarise to ice

## TRANSFORM THE DATA ##
# Data Transformer
#tensor_sar = sar.ToTensor()
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    #transforms.unsqeueeze(0)

# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
tensor_sar = transform(sar)

## DEFINE THE CLASSIFICATION CLASSES ##


# To fix the following error: "URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>" #
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## DEFINE THE MODEL ##
#class Model (pl.LightningModule):
#    densenet = smp.Unet('densenet201', encoder_weights='imagenet', in_channels=3)
#    # Replace Output of Fully Connected Layer with Number of Labels for our Classification Problem
#    densenet.fc = Linear(in_features=512, out_features=2)
#    # Optimiser
#    # TEST BOTH - optimiser1 doesn't work
#    # optimiser1 = Adam(densenet.parameters(), lr=3e-4, weight_decay=0.0001)
#    optimizer = torch.optim.Adam(densenet.parameters(), lr=3e-4)
#
#    # Loss Function
#    loss_fn = nn.CrossEntropyLoss()

### CORRECTED CLASS DEFINITION ###
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.densenet = smp.Unet('densenet201', encoder_weights='imagenet', in_channels=3)
        # Replace output of Fully Connected Layer with number of labels for our classification problem
        self.densenet.classifier = nn.Linear(in_features=512, out_features=2)
        # Optimizer
        self.optimizer = torch.optim.Adam(self.densenet.parameters(), lr=3e-4)
        # Loss Function
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        # Define how the input x should pass through the layers of the model
        x = self.densenet(x)
        return x

    # TO FIX THE ERROR MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.
    def training_step(self, batch, batch_idx):
        # Define what happens during one training step on one batch of data
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y_pred, y)
        self.log('train_loss', loss)
        return loss

    # TO FIX THE ERROR MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.
    def configure_optimizers(self):
        # Return a list of optimizers and LR schedulers to use in training
        return self.optimizer

    # TO FIX THE ERROR MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.
    def train_dataloader(self):
        # Return a DataLoader object for the training data
        # Replace with your own DataLoader object
        return train_dataloader

## TRAIN THE MODEL ##
model = Model()

# Instantiate a Trainer object and train the model
trainer = Trainer(
    gpus=0,
    accelerator="cpu", 
    max_epochs=10 
    #callbacks=[early_stop_callback], 
    #progress_bar_refresh_rate=20
)

trainer.fit(model, tensor_sar)

#model.fit(model, tensor_sar) 

## TEST THE MODEL ##


In [ ]:
## IMPORT THE MODULES ##
import os
import torch
import numpy as np
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from skimage import io
from torchvision import transforms
import torch.nn as nn
from pytorch_lightning import Trainer

#### LOAD THE DATA : ATTEMPT 3 ###
sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


sar = io.imread(sar_name).copy()  # take all bands for shape of 256 x 256 x 3
chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
chart[chart < 80] = 0  # binarise to water
chart[chart >= 80] = 255  # binarise to ice

## TRANSFORM THE DATA ##
# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
tensor_sar = transform(sar)

# To fix the following error: "URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>" #
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## DEFINE THE MODEL ##
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.densenet = smp.Unet('densenet201', encoder_weights='imagenet', in_channels=3)
        # Replace output of Fully Connected Layer with number of labels for our classification problem
        self.densenet.classifier = nn.Linear(in_features=512, out_features=2)
        # Optimizer
        self.optimizer = torch.optim.Adam(self.densenet.parameters(), lr=3e-4)
        # Loss Function
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        # Define how the input x should pass through the layers of the model
        x = self.densenet(x)
        return x

    ### THE "TOO MANY VALUES ERROR CONTINUED, even after updating train_dataloader."
    # Define what happens during one training step on one batch of data
    #def training_step(self, batch, batch_idx):
    #    x, y = batch
    #    y_pred = self.forward(x)
    #    loss = self.loss_fn(y_pred, y)
    #    self.log('train_loss', loss)
    #    return loss

    def training_step(self, batch, batch_idx):
    # Define what happens during one training step on one batch of data
        x, y, *_ = batch  # use * to unpack the remaining elements in batch if there are more than 2
        y_pred = self.forward(x)
        loss = self.loss_fn(y_pred, y)
        self.log('train_loss', loss)
        return loss

    # Return a list of optimizers and LR schedulers to use in training
    def configure_optimizers(self):
        return self.optimizer

    def train_dataloader(self):
    # Replace with your own DataLoader object
        # Updating train_data to a list of tuples where each tuple has the SAR tensor and the corresponding chart tensor
        train_data = [(tensor_sar, chart.unsqueeze(0))]
        train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
        for batch in train_loader:
            yield batch[0], batch[1].unsqueeze(1).long()

    ### THIS ALSO DID NOT WORK - ValueError: not enough values to unpack (expected 2, got 1) ###
    #def train_dataloader(self):
    ## Replace with your own DataLoader object
    #    train_data = [(tensor_sar, chart)]
    #    train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
    #    return train_loader
    
    ### THIS DID NOT WORK - ValueError: too many values to unpack (expected 2) ###
    ## Return a DataLoader object for the training data
    #def train_dataloader(self):
    #    return torch.utils.data.DataLoader(tensor_sar, batch_size=1)

## TRAIN THE MODEL ##
model = Model()

# Instantiate a Trainer object and train the model
trainer = Trainer(
    gpus=0,
    accelerator="cpu", 
    max_epochs=10 
    #callbacks=[early_stop_callback],
    )

trainer.fit(model, tensor_sar)


In [ ]:
import torch.utils.data as data_utils

# define the training data
train_sar = tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
train_chart = chart.unsqueeze(0)  # add a batch dimension to chart
train_data = data_utils.TensorDataset(train_sar, train_chart)

# define the data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

# update train_dataloader method
def train_dataloader(self):
    return train_loader

# update training_step method
def training_step(self, batch, batch_idx):
    x, y = batch
    y_pred = self.forward(x)
    loss = self.loss_fn(y_pred, y.squeeze(1).long())
    self.log('train_loss', loss)
    return loss


In [61]:
## IMPORT THE MODULES ##
import os
import torch
import numpy as np
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from skimage import io
from torchvision import transforms
import torch.nn as nn
from pytorch_lightning import Trainer
import torch.utils.data as data_utils


#### LOAD THE DATA : ATTEMPT 3 ###
sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


### THIS CODE FAILS WITH ERROR RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead #
#sar = io.imread(sar_name).copy()  # take all bands for shape of 256 x 256 x 3
## To solve the error: RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead #
## This code checks if the SAR data has only 1 channel and, if so, replicates the single channel 3 times to make it have 3 channels. This should ensure that the input to the model has the correct number of channels.
#if sar.shape[-1] == 1:
#    sar = np.repeat(sar, 3, axis=-1)
#chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
#chart[chart < 80] = 0  # binarise to water
#chart[chart >= 80] = 255  # binarise to ice

### TRY AGAIN ###
sar = io.imread(sar_name).copy()[:, :, np.newaxis]  # add a channel dimension for shape of 256 x 256 x 1
sar = np.repeat(sar, 3, axis=2)  # repeat the same SAR data to create 3-channel image
chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
chart[chart < 80] = 0  # binarise to water
chart[chart >= 80] = 255  # binarise to ice


## TRANSFORM THE DATA ##
# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
tensor_sar = transform(sar)
tensor_chart = transform(chart)

# define the training data
train_sar = tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
train_chart = tensor_chart.unsqueeze(0)  # add a batch dimension to chart
train_data = data_utils.TensorDataset(train_sar, train_chart)

# define the data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

# To fix the following error: "URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>" #
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## DEFINE THE MODEL ##
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.densenet = smp.Unet('densenet201', encoder_weights='imagenet', in_channels=3)
        # Replace output of Fully Connected Layer with number of labels for our classification problem
        self.densenet.classifier = nn.Linear(in_features=512, out_features=2)
        # Optimizer
        self.optimizer = torch.optim.Adam(self.densenet.parameters(), lr=3e-4)
        # Loss Function
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        # Define how the input x should pass through the layers of the model
        x = self.densenet(x)
        return x

    ### THE "TOO MANY VALUES ERROR CONTINUED, even after updating train_dataloader."
    # Define what happens during one training step on one batch of data
    #def training_step(self, batch, batch_idx):
    #    x, y = batch
    #    y_pred = self.forward(x)
    #    loss = self.loss_fn(y_pred, y)
    #    self.log('train_loss', loss)
    #    return loss

    def training_step(self, batch, batch_idx):
    # Define what happens during one training step on one batch of data
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y_pred, y.squeeze(1).long())
        self.log('train_loss', loss)
        return loss

    # Return a list of optimizers and LR schedulers to use in training
    def configure_optimizers(self):
        return self.optimizer

    def train_dataloader(self):
    # Replace with your own DataLoader object
        train_data = [(tensor_sar, tensor_chart)]
        input_data = torch.stack([data[0] for data in train_data])
        target_data = torch.stack([data[1] for data in train_data])
        train_loader = torch.utils.data.DataLoader(list(zip(input_data, target_data)), batch_size=1, shuffle=True)
        return train_loader
    
    ### THIS RETURNS ERROR ValueError: too many values to unpack (expected 2) ###
    #def train_dataloader(self):
    #    return train_loader

    ### THIS ALSO DID NOT WORK - ValueError: not enough values to unpack (expected 2, got 1) ###
    #def train_dataloader(self):
    ## Replace with your own DataLoader object
    #    train_data = [(tensor_sar, chart)]
    #    train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
    #    return train_loader
    
    ### THIS DID NOT WORK - ValueError: too many values to unpack (expected 2) ###
    ## Return a DataLoader object for the training data
    #def train_dataloader(self):
    #    return torch.utils.data.DataLoader(tensor_sar, batch_size=1)

## TRAIN THE MODEL ##
model = Model()

# Instantiate a Trainer object and train the model
trainer = Trainer(
    gpus=0,
    accelerator="cpu", 
    max_epochs=10 
    #callbacks=[early_stop_callback],
    )

trainer.fit(model, train_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type             | Params
----------------------------------------------
0 | densenet | Unet             | 28.6 M
1 | loss_fn  | CrossEntropyLoss | 0     
----------------------------------------------
28.6 M    Trainable params
0         Non-trainable params
28.6 M    Total params
114.326   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/1 [12:03<?, ?it/s]


`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:03<00:00,  3.53s/it, loss=0, v_num=11]


In [58]:
print(sar.shape)

(256, 256)


In [59]:
print(sar.shape[-1])

256


# OMG THIS WORKS
DO NOT TOUCH

In [62]:
## IMPORT THE MODULES ##
import os
import torch
import numpy as np
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from skimage import io
from torchvision import transforms
import torch.nn as nn
from pytorch_lightning import Trainer
import torch.utils.data as data_utils


#### LOAD THE DATA : ATTEMPT 3 ###
sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


### THIS CODE FAILS WITH ERROR RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead #
#sar = io.imread(sar_name).copy()  # take all bands for shape of 256 x 256 x 3
## To solve the error: RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead #
## This code checks if the SAR data has only 1 channel and, if so, replicates the single channel 3 times to make it have 3 channels. This should ensure that the input to the model has the correct number of channels.
#if sar.shape[-1] == 1:
#    sar = np.repeat(sar, 3, axis=-1)
#chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
#chart[chart < 80] = 0  # binarise to water
#chart[chart >= 80] = 255  # binarise to ice

### TRY AGAIN ###
sar = io.imread(sar_name).copy()[:, :, np.newaxis]  # add a channel dimension for shape of 256 x 256 x 1
sar = np.repeat(sar, 3, axis=2)  # repeat the same SAR data to create 3-channel image
chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
chart[chart < 80] = 0  # binarise to water
chart[chart >= 80] = 255  # binarise to ice


## TRANSFORM THE DATA ##
# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
tensor_sar = transform(sar)
tensor_chart = transform(chart)

# define the training data
train_sar = tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
train_chart = tensor_chart.unsqueeze(0)  # add a batch dimension to chart
train_data = data_utils.TensorDataset(train_sar, train_chart)

# define the data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

# To fix the following error: "URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>" #
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## DEFINE THE MODEL ##
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.densenet = smp.Unet('densenet201', encoder_weights='imagenet', in_channels=3)
        # Replace output of Fully Connected Layer with number of labels for our classification problem
        self.densenet.classifier = nn.Linear(in_features=512, out_features=2)
        # Optimizer
        self.optimizer = torch.optim.Adam(self.densenet.parameters(), lr=3e-4)
        # Loss Function
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        # Define how the input x should pass through the layers of the model
        x = self.densenet(x)
        return x

    ### THE "TOO MANY VALUES ERROR CONTINUED, even after updating train_dataloader."
    # Define what happens during one training step on one batch of data
    #def training_step(self, batch, batch_idx):
    #    x, y = batch
    #    y_pred = self.forward(x)
    #    loss = self.loss_fn(y_pred, y)
    #    self.log('train_loss', loss)
    #    return loss

    def training_step(self, batch, batch_idx):
    # Define what happens during one training step on one batch of data
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y_pred, y.squeeze(1).long())
        self.log('train_loss', loss)
        return loss

    # Return a list of optimizers and LR schedulers to use in training
    def configure_optimizers(self):
        return self.optimizer

    def train_dataloader(self):
    # Replace with your own DataLoader object
        train_data = [(tensor_sar, tensor_chart)]
        input_data = torch.stack([data[0] for data in train_data])
        target_data = torch.stack([data[1] for data in train_data])
        train_loader = torch.utils.data.DataLoader(list(zip(input_data, target_data)), batch_size=1, shuffle=True)
        return train_loader
    
    ### THIS RETURNS ERROR ValueError: too many values to unpack (expected 2) ###
    #def train_dataloader(self):
    #    return train_loader

    ### THIS ALSO DID NOT WORK - ValueError: not enough values to unpack (expected 2, got 1) ###
    #def train_dataloader(self):
    ## Replace with your own DataLoader object
    #    train_data = [(tensor_sar, chart)]
    #    train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
    #    return train_loader
    
    ### THIS DID NOT WORK - ValueError: too many values to unpack (expected 2) ###
    ## Return a DataLoader object for the training data
    #def train_dataloader(self):
    #    return torch.utils.data.DataLoader(tensor_sar, batch_size=1)

## TRAIN THE MODEL ##
model = Model()

# Instantiate a Trainer object and train the model
trainer = Trainer(
    gpus=0,
    accelerator="cpu", 
    max_epochs=10 
    #callbacks=[early_stop_callback],
    )

trainer.fit(model, train_loader)

# Save the trained model to a file
trainer.save_checkpoint("/Users/meghanplumridge/Desktop/checkpoint.ckpt")


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type             | Params
----------------------------------------------
0 | densenet | Unet             | 28.6 M
1 | loss_fn  | CrossEntropyLoss | 0     
----------------------------------------------
28.6 M    Trainable params
0         Non-trainable params
28.6 M    Total params
114.326   Total estimated model params size (MB)


Epoch 9: 100%|██████████| 1/1 [00:02<00:00,  2.69s/it, loss=0, v_num=12]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:03<00:00,  3.92s/it, loss=0, v_num=12]


# WORKING, BOTH TRAIN AND TEST
DO NOT MODIFY

In [65]:
## IMPORT THE MODULES ##
import os
import torch
import numpy as np
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from skimage import io
from torchvision import transforms
import torch.nn as nn
from pytorch_lightning import Trainer
import torch.utils.data as data_utils

####################################################################################

#### LOAD THE TRAINING DATA : ATTEMPT 3 ###
sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


### THIS CODE FAILS WITH ERROR RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead #
#sar = io.imread(sar_name).copy()  # take all bands for shape of 256 x 256 x 3
## To solve the error: RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead #
## This code checks if the SAR data has only 1 channel and, if so, replicates the single channel 3 times to make it have 3 channels. This should ensure that the input to the model has the correct number of channels.
#if sar.shape[-1] == 1:
#    sar = np.repeat(sar, 3, axis=-1)
#chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
#chart[chart < 80] = 0  # binarise to water
#chart[chart >= 80] = 255  # binarise to ice

### TRY AGAIN ###
sar = io.imread(sar_name).copy()[:, :, np.newaxis]  # add a channel dimension for shape of 256 x 256 x 1
sar = np.repeat(sar, 3, axis=2)  # repeat the same SAR data to create 3-channel image
chart = io.imread(chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
chart[chart < 80] = 0  # binarise to water
chart[chart >= 80] = 255  # binarise to ice


## TRANSFORM THE DATA ##
# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
tensor_sar = transform(sar)
tensor_chart = transform(chart)

# define the training data
train_sar = tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
train_chart = tensor_chart.unsqueeze(0)  # add a batch dimension to chart
train_data = data_utils.TensorDataset(train_sar, train_chart)

# define the data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

####################################################################################

#### LOAD THE TEST DATA ####

## LOAD THE TEST DATA ##
test_sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
test_chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


test_sar = io.imread(test_sar_name).copy()[:, :, np.newaxis]  # add a channel dimension for shape of 256 x 256 x 1
test_sar = np.repeat(test_sar, 3, axis=2)  # repeat the same SAR data to create 3-channel image
test_chart = io.imread(test_chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
test_chart[test_chart < 80] = 0  # binarise to water
test_chart[test_chart >= 80] = 255  # binarise to ice

# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
test_tensor_sar = transform(test_sar)
test_tensor_chart = transform(test_chart)

# define the test data
test_sar = test_tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
test_chart = test_tensor_chart.unsqueeze(0)  # add a batch dimension to chart
test_data = data_utils.TensorDataset(test_sar, test_chart)

# define the data loader
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

####################################################################################

# To fix the following error: "URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>" #
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## DEFINE THE MODEL ##
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.densenet = smp.Unet('densenet201', encoder_weights='imagenet', in_channels=3)
        # Replace output of Fully Connected Layer with number of labels for our classification problem
        self.densenet.classifier = nn.Linear(in_features=512, out_features=2)
        # Optimizer
        self.optimizer = torch.optim.Adam(self.densenet.parameters(), lr=3e-4)
        # Loss Function
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        # Define how the input x should pass through the layers of the model
        x = self.densenet(x)
        return x

    ### THE "TOO MANY VALUES ERROR CONTINUED, even after updating train_dataloader."
    # Define what happens during one training step on one batch of data
    #def training_step(self, batch, batch_idx):
    #    x, y = batch
    #    y_pred = self.forward(x)
    #    loss = self.loss_fn(y_pred, y)
    #    self.log('train_loss', loss)
    #    return loss

    def training_step(self, batch, batch_idx):
    # Define what happens during one training step on one batch of data
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y_pred, y.squeeze(1).long())
        self.log('train_loss', loss)
        return loss

    # Return a list of optimizers and LR schedulers to use in training
    def configure_optimizers(self):
        return self.optimizer

    def train_dataloader(self):
    # Replace with your own DataLoader object
        train_data = [(tensor_sar, tensor_chart)]
        input_data = torch.stack([data[0] for data in train_data])
        target_data = torch.stack([data[1] for data in train_data])
        train_loader = torch.utils.data.DataLoader(list(zip(input_data, target_data)), batch_size=1, shuffle=True)
        return train_loader

    def test_step(self, batch, batch_idx):
        # Define what happens during one test step on one batch of data
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y_pred, y.squeeze(1).long())
        self.log('test_loss', loss)
        return loss

    def test_dataloader(self):
        # Replace with your own DataLoader object
        return test_loader
    
    ### THIS RETURNS ERROR ValueError: too many values to unpack (expected 2) ###
    #def train_dataloader(self):
    #    return train_loader

    ### THIS ALSO DID NOT WORK - ValueError: not enough values to unpack (expected 2, got 1) ###
    #def train_dataloader(self):
    ## Replace with your own DataLoader object
    #    train_data = [(tensor_sar, chart)]
    #    train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
    #    return train_loader
    
    ### THIS DID NOT WORK - ValueError: too many values to unpack (expected 2) ###
    ## Return a DataLoader object for the training data
    #def train_dataloader(self):
    #    return torch.utils.data.DataLoader(tensor_sar, batch_size=1)

## TRAIN THE MODEL ##
model = Model()

# Instantiate a Trainer object and train the model
trainer = Trainer(
    gpus=0,
    accelerator="cpu", 
    max_epochs=10 
    #callbacks=[early_stop_callback],
    )

trainer.fit(model, train_loader)

# Save the trained model to a file
trainer.save_checkpoint("/Users/meghanplumridge/Desktop/checkpoint.ckpt")

# Run the model
trainer.test(model, test_loader)

/Users/meghanplumridge/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type             | Params
----------------------------------------------
0 | densenet | Unet             | 28.6 M
1 | loss_fn  | CrossEntropyLoss | 0     
----------------------------------------------
28.6 M    Trainable params
0         Non-trainable params
28.6 M    Total params
114.326   Total estimated model params size (MB)
/Users/meghanplumridge/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not

Epoch 9: 100%|██████████| 1/1 [00:01<00:00,  1.22s/it, loss=0, v_num=13]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:01<00:00,  1.86s/it, loss=0, v_num=13]


/Users/meghanplumridge/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.77it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss                   0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0}]

In [64]:
### RUN THE MODEL WITH TEST DATA ###

## LOAD THE TEST DATA ##
test_sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
test_chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


test_sar = io.imread(test_sar_name).copy()[:, :, np.newaxis]  # add a channel dimension for shape of 256 x 256 x 1
test_sar = np.repeat(test_sar, 3, axis=2)  # repeat the same SAR data to create 3-channel image
test_chart = io.imread(test_chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
test_chart[test_chart < 80] = 0  # binarise to water
test_chart[test_chart >= 80] = 255  # binarise to ice

# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
test_tensor_sar = transform(test_sar)
test_tensor_chart = transform(test_chart)

# define the test data
test_sar = test_tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
test_chart = test_tensor_chart.unsqueeze(0)  # add a batch dimension to chart
test_data = data_utils.TensorDataset(test_sar, test_chart)

# define the data loader
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)


trainer.test(model, test_loader)


MisconfigurationException: No `test_step()` method defined to run `Trainer.test`.

In [63]:
### RUN THE MODEL WITH TEST DATA ###

## LOAD THE TEST DATA ##
test_sar_name = "/Users/meghanplumridge/Desktop/CNN_sar_data/lbls_WS_20181202T_00088_[10752,1280]_256x256.tiff"
test_chart_name = "/Users/meghanplumridge/Desktop/CNN_chart_data/ftrs_WS_20181202T_00088_[10752,1280]_256x256.tiff"


test_sar = io.imread(test_sar_name).copy()[:, :, np.newaxis]  # add a channel dimension for shape of 256 x 256 x 1
test_sar = np.repeat(test_sar, 3, axis=2)  # repeat the same SAR data to create 3-channel image
test_chart = io.imread(test_chart_name).copy()[:, :, 0]  # take red band only for shape of 256 x 256 x 1
test_chart[test_chart < 80] = 0  # binarise to water
test_chart[test_chart >= 80] = 255  # binarise to ice

# Convert SAR data to PyTorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
test_tensor_sar = transform(test_sar)
test_tensor_chart = transform(test_chart)

# define the test data
test_sar = test_tensor_sar.unsqueeze(0)  # add a batch dimension to tensor_sar
test_chart = test_tensor_chart.unsqueeze(0)  # add a batch dimension to chart
test_data = data_utils.TensorDataset(test_sar, test_chart)

# define the data loader
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)


## LOAD THE TRAINED MODEL ##
model = Model.load_from_checkpoint("/Users/meghanplumridge/Desktop/checkpoint.ckpt")

## RUN THE PREDICTIONS ##
predictions = []
for batch in test_loader:
    # Make predictions on the test batch
    pred = model.predict(batch)
    # Append the predictions to the list of predictions
    predictions.append(pred)

# Convert the predictions to a numpy array
predictions = np.concatenate(predictions, axis=0)

# You can now use the predictions as desired


AttributeError: 'Model' object has no attribute 'predict'